# Model Output Notebook

<img style="float:center;" src="https://arcticexpansion.vse.gmu.edu/sites/arcticexpansion.vsnet.gmu.edu/files/images/header5d2.png" width=600px>

### ADCIRC-SWAN Output


### Initialize Libraries

In [1]:
import netCDF4 as nc4;
import pathlib as pl;     
import numpy as np   

source $HOME/miniforge3/bin/activate

salloc --ntasks=5 --nodes=1 --partition=normal --time=10:00:00

### Defined Functions

#### Data for this exercise can be found here
https://doi.org/10.17603/ds2-h0fw-2p96

Download the swan_HS.63.nc from one of the 4 folders

---

In [2]:
INPUT_ROOT = pl.Path('/groups/ORC-CLIMATE/fhrl_repo/Arctic_Database/Raw_DATA')
OUTPUT_ROOT = pl.Path('/scratch/tmiesse/project/data4spatial')


In [ ]:
YEARS       = [2023]#2021,2020,2019,2018,2017,2016,2015,
               #2014,2013,2012,2011,2010,2009,2008,
               #2007,2006,2005,2004,2003,2002,2001,
               #2000,1999,1998,1997,1996,1995,1994,
               #1993,1992,1991,1990]
KEEP_VARS   = ["time", "x", "y", "element", "zeta", "depth"]

# depth bounds
MIN_DEPTH = 0.0
MAX_DEPTH = 5.0

for year in YEARS:
    src_path = INPUT_ROOT / str(year) / "outputs" / "fort.63.nc"
    if not src_path.exists():
        print(f"Skipping {year}: {src_path} not found")
        continue

    out_dir  = OUTPUT_ROOT / str(year)
    out_dir.mkdir(parents=True, exist_ok=True)
    dst_path = out_dir / "fort.63.cf.nc"
    print(f"Processing {year} → {dst_path}")

    with nc4.Dataset(src_path, "r") as ds0:
        # 0) pick off the depth array & compute keep_idx
        depth_arr = ds0.variables["depth"][:]  # (node,)
        fv_depth  = getattr(ds0.variables["depth"], "_FillValue", None)
        # build mask of valid nodes
        mask = np.ones_like(depth_arr, dtype=bool)
        if fv_depth is not None:
            mask &= (depth_arr != fv_depth)
        mask &= (depth_arr >= MIN_DEPTH) & (depth_arr <= MAX_DEPTH)
        keep_idx = np.nonzero(mask)[0]
        n_node_new = keep_idx.size

        # 1) read & filter connectivity (element → face_node_connectivity)
        conn0 = ds0.variables["element"][:]            # shape (nele, nvertex)
        # find triangles fully in shallow nodes
        tri_mask = np.all(np.isin(conn0, keep_idx), axis=1)
        conn_filt = conn0[tri_mask, :]
        # build map global->local
        local_map = {g: i for i, g in enumerate(keep_idx)}
        remap = np.vectorize(local_map.get)(conn_filt)
        n_nele_new = remap.shape[0]

        # 2) start new CF file
        with nc4.Dataset(dst_path, "w") as ds1:
            # 1) copy global attributes
            ds1.setncatts({k: ds0.getncattr(k) for k in ds0.ncattrs()})

            # 2) create dimensions
            for name, dim in ds0.dimensions.items():
                if name == "node":
                    ds1.createDimension("node", n_node_new)
                elif name == "nele":
                    ds1.createDimension("nele", n_nele_new)
                else:
                    size = None if dim.isunlimited() else len(dim)
                    ds1.createDimension(name, size)

            # 3) copy all other variables except 'element'
            KEEP_VARS = ["time", "x", "y", "zeta", "depth"]
            for name in KEEP_VARS:
                var0 = ds0.variables[name]
                dims = var0.dimensions
                fv   = getattr(var0, "_FillValue", None)

                # create var in ds1
                if fv is not None:
                    var1 = ds1.createVariable(name, var0.datatype, dims, fill_value=fv)
                else:
                    var1 = ds1.createVariable(name, var0.datatype, dims)

                # copy data, slicing on node if needed
                if "node" in dims:
                    idx = tuple(
                        keep_idx if d == "node" else slice(None)
                        for d in dims
                    )
                    var1[:] = var0[idx]
                else:
                    var1[:] = var0[:]

                # copy attributes (except _FillValue)
                for attr, val in var0.__dict__.items():
                    if attr == "_FillValue":
                        continue
                    var1.setncattr(attr, val)

            # 4) now write the filtered & remapped connectivity
            conn_var = ds1.createVariable(
                "face_node_connectivity",
                remap.dtype,
                ("nele", "nvertex")
            )
            conn_var[:] = remap
            # copy attrs from original element var
            for attr, val in ds0.variables["element"].__dict__.items():
                if attr == "_FillValue":
                    continue
                conn_var.setncattr(attr, val)
            conn_var.setncattr("cf_role", "face_node_connectivity")

            # 5) define mesh topology
            mesh = ds1.createVariable("mesh2d", "i4", ())
            mesh.setncattr("cf_role", "mesh_topology")
            mesh.setncattr("topology_dimension", 2)
            mesh.setncattr("node_coordinates", "x y")
            mesh.setncattr("face_node_connectivity", "face_node_connectivity")

    print(f"  → Done {year}")

print("All years processed; CF‑ified, depth‑filtered files under:", OUTPUT_ROOT)

Processing 2024 → /scratch/tmiesse/project/data4spatial/2024/fort.63.cf.nc


In [9]:
YEARS = [2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,
         2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,
         1999,1998,1997,1996,1995,1994,1993,1992,1991,1990]
KEEP_VARS = ["time", "x", "y", "element", "zeta", "depth"]
for year in YEARS:
    src_path = INPUT_ROOT / str(year) / 'outputs' / 'fort.63.nc'
    if not src_path.exists():
        print(f"Skipping {year}: {src_path} not found")
        continue

    # Prepare output directory for this year
    out_dir = OUTPUT_ROOT / str(year)
    out_dir.mkdir(parents=True, exist_ok=True)
    dst_path = out_dir / 'fort.63.cf.nc'

    print(f"Processing year {year} → {dst_path}")

    with nc4.Dataset(src_path, 'r') as ds0, nc4.Dataset(dst_path, 'w') as ds1:
        # 1) Copy global attributes
        ds1.setncatts({k: ds0.getncattr(k) for k in ds0.ncattrs()})

        # 2) Copy dimensions
        for name, dim in ds0.dimensions.items():
            ds1.createDimension(name, len(dim) if not dim.isunlimited() else None)

        # 3) Copy selected variables, handling _FillValue
        for name in KEEP_VARS:
            var0 = ds0.variables[name]
            out_name = "face_node_connectivity" if name == "element" else name

            # Determine fill_value if present
            fv = getattr(var0, "_FillValue", None)

            # Create variable with fill_value if needed
            if fv is not None:
                var1 = ds1.createVariable(out_name,
                                          var0.datatype,
                                          var0.dimensions,
                                          fill_value=fv)
            else:
                var1 = ds1.createVariable(out_name,
                                          var0.datatype,
                                          var0.dimensions)

            # Copy the data
            var1[:] = var0[:]

            # Copy all attributes except _FillValue
            for attr, val in var0.__dict__.items():
                if attr == "_FillValue":
                    continue
                var1.setncattr(attr, val)

            # Mark connectivity variable
            if name == "element":
                var1.setncattr("cf_role", "face_node_connectivity")

        # 4) Declare the mesh topology
        mesh = ds1.createVariable("mesh2d", "i4", ())
        mesh.setncattr("cf_role", "mesh_topology")
        mesh.setncattr("topology_dimension", 2)
        mesh.setncattr("node_coordinates", "x y")
        mesh.setncattr("face_node_connectivity", "face_node_connectivity")

    print(f"  → Done {year}")

print("All years processed. CF‑ified files are under:", OUTPUT_ROOT)

Processing year 2021 → /scratch/tmiesse/project/data4spatial/2021/fort.63.cf.nc
  → Done 2021
Processing year 2020 → /scratch/tmiesse/project/data4spatial/2020/fort.63.cf.nc
  → Done 2020
Processing year 2019 → /scratch/tmiesse/project/data4spatial/2019/fort.63.cf.nc
  → Done 2019
Processing year 2018 → /scratch/tmiesse/project/data4spatial/2018/fort.63.cf.nc
  → Done 2018
Processing year 2017 → /scratch/tmiesse/project/data4spatial/2017/fort.63.cf.nc
  → Done 2017
Processing year 2016 → /scratch/tmiesse/project/data4spatial/2016/fort.63.cf.nc
  → Done 2016
Processing year 2015 → /scratch/tmiesse/project/data4spatial/2015/fort.63.cf.nc
  → Done 2015
Processing year 2014 → /scratch/tmiesse/project/data4spatial/2014/fort.63.cf.nc
  → Done 2014
Processing year 2013 → /scratch/tmiesse/project/data4spatial/2013/fort.63.cf.nc
  → Done 2013
Processing year 2012 → /scratch/tmiesse/project/data4spatial/2012/fort.63.cf.nc
  → Done 2012
Processing year 2011 → /scratch/tmiesse/project/data4spatial